In [1]:
#!/usr/bin/env python

import numpy as np, os, sys
from scipy.io import loadmat
import joblib
from run_12ECG_classifier import load_12ECG_model, run_12ECG_classifier
from driver import get_classes, load_challenge_data
from get_12ECG_features import get_12ECG_features
sys.path.append(os.path.abspath("./datasets"))
# sys.path.append(os.path.abspath("../ecg/ecg/")) # update
sys.path.append(os.path.abspath("./evaluation-2020/"))
sys.path.append(os.path.abspath("./models/"))
sys.path.append(os.path.abspath("./util/"))

from helper import translate_x, translate_y, get_data_from_physionet2020Dataset
from physionet2020 import PhysioNet2020Dataset
from torch.utils.data import DataLoader
import time
import tensorflow as tf

import evaluate_12ECG_score
import argparse
import json
import keras
import random
import network_tf
# import load
import util
import tqdm
from datetime import datetime

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use id from $ nvidia-smi
tf.test.is_gpu_available()

In [4]:
# for tensorboard
%load_ext tensorboard
from tensorflow import keras

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

In [46]:
train_records, val_records = PhysioNet2020Dataset.split_names_cv(
    "Training_WFDB", 5, 0
)

dev_x, dev_y = get_data_from_physionet2020Dataset(val_records)
train_x, train_y = get_data_from_physionet2020Dataset(train_records)
train_y = np.concatenate((train_y[:, :3], train_y[:, 4:]), axis=1)
dev_y = np.concatenate((dev_y[:, :3], dev_y[:, 4:]), axis=1)

(3497, 9)
(3497, 4096, 12)
(3497, 9)
(13946, 9)
(13946, 4096, 12)
(13946, 9)


In [7]:
def epoch_evaluation(probs, dev_y):

    threshold = 0.5

    probs_class = probs
    probs_class[probs_class > threshold] = 0.99
    probs_class[probs_class <= threshold] = 0.01
    probs_test = probs_class

    sys.path.append(os.path.abspath("./evaluation-2020/"))
    import evaluate_12ECG_score
    auroc,auprc = evaluate_12ECG_score.compute_auc(dev_y, probs, 8)

    probs_class = probs
    probs_class[probs_class > threshold] = 1
    probs_class[probs_class <= threshold] = 0
    probs_test = probs_class
    accuracy,f_measure,f_beta,g_beta = evaluate_12ECG_score.compute_beta_score(dev_y, probs_test, 2, 8)
    print ([auroc,auprc,accuracy,f_measure,f_beta,g_beta])
    
    return {
        'auroc': auroc,
        'auprc': auprc,
        'accuracy': accuracy,
        'f_measure': f_measure,
        'f_beta': f_beta,
        'g_beta': g_beta
    }

class validationDataCallback(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(validationDataCallback, self).__init__()
        self.valid_data = valid_data
    
    def on_epoch_end(self, epoch, logs=None):
        x_test = self.valid_data[0]
        y_test = self.valid_data[1]
        predictions = self.model.predict(x_test, batch_size=32, verbose=1)
        epoch_data = epoch_evaluation(predictions, y_test)
        for key, value in epoch_data.items():
            tf.summary.scalar(key, data=value, step=epoch)


In [8]:
from tensorflow.keras.models import load_model
from automodel import model
from tensorflow.keras.callbacks import (ModelCheckpoint,
                             TensorBoard, ReduceLROnPlateau,
                             CSVLogger, EarlyStopping)
from tensorflow.keras.optimizers import Adam
# model_6labels = load_model('../automatic-ecg-diagnosis/model.hdf5', compile=False)

# weight = model_6labels.get_weights()

# weight[-2] = np.hstack([weight[-2], np.random.randint(2, size=[weight[-2].shape[0], 2])])
# bias = np.zeros(8)
# bias[:6] = weight[-1]
# weight[-1] = bias

# model.set_weights(weight)

In [11]:
loss = 'binary_crossentropy'
lr = 0.001
batch_size = 64
opt = Adam(lr)

callbacks = [ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=7,
                               min_lr=lr / 100),
             EarlyStopping(patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                           min_delta=0.00001),
             validationDataCallback([dev_x, dev_y])]

In [17]:
# Train neural network
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.fit(train_x, train_y,
        batch_size=batch_size,
        epochs=70,
        initial_epoch=0,  # If you are continuing a interrupted section change here
        validation_split=0.2,
        shuffle='batch',  # Because our dataset is an HDF5 file
        callbacks=callbacks,
        verbose=1)

Train on 11156 samples, validate on 2790 samples
Epoch 1/70
3497/3497 [==============================] - 2s 494us/sample


/home/weijiesun/ecg/physionet-challenge-2020/evaluation-2020/evaluate_12ECG_score.py:215: RuntimeWarning: divide by zero encountered in true_divide
  tn += 1/num_labels


[0.49995571098834096, 0.11870889333714613, nan, 0.0, 0.0, 0.0]
11156/11156 [==============================] - 24s 2ms/sample - loss: 0.4745 - accuracy: 0.8594 - val_loss: 0.4220 - val_accuracy: 0.8810
Epoch 2/70
3497/3497 [==============================] - 2s 438us/sample
[0.49997993579454253, 0.11870889333714613, nan, 0.0, 0.0, 0.0]
11156/11156 [==============================] - 21s 2ms/sample - loss: 0.4379 - accuracy: 0.8729 - val_loss: 0.4221 - val_accuracy: 0.8810
Epoch 3/70
3497/3497 [==============================] - 2s 460us/sample
[0.5001564304675907, 0.11889070148271824, nan, 0.00015163768701981396, 9.516317312084452e-05, 3.803631199918855e-05]
11156/11156 [==============================] - 21s 2ms/sample - loss: 0.4218 - accuracy: 0.8753 - val_loss: 0.3817 - val_accuracy: 0.8809
Epoch 4/70
   64/11156 [..............................] - ETA: 17s - loss: 0.4794 - accuracy: 0.8652

/home/weijiesun/ecg/physionet-challenge-2020/evaluation-2020/evaluate_12ECG_score.py:211: RuntimeWarning: divide by zero encountered in true_divide
  fp += 1/num_labels


3497/3497 [==============================] - 2s 460us/sample
[0.5001166368077306, 0.1187779908528851, nan, 0.0, 0.0, 0.0]
11156/11156 [==============================] - 20s 2ms/sample - loss: 0.4103 - accuracy: 0.8763 - val_loss: 0.3863 - val_accuracy: 0.8810
Epoch 5/70
3497/3497 [==============================] - 2s 474us/sample
[0.499915582577426, 0.11870889333714613, nan, 0.0, 0.0, 0.0]
11156/11156 [==============================] - 20s 2ms/sample - loss: 0.4006 - accuracy: 0.8769 - val_loss: 0.3813 - val_accuracy: 0.8809
Epoch 6/70
3497/3497 [==============================] - 1s 427us/sample
[0.5001072150925567, 0.11878401614844193, nan, 0.0009048863864870298, 0.0005701976685250886, 0.00022828733766233763]
11156/11156 [==============================] - 19s 2ms/sample - loss: 0.3924 - accuracy: 0.8780 - val_loss: 0.3783 - val_accuracy: 0.8808
Epoch 7/70
3497/3497 [==============================] - 2s 444us/sample
[0.49990115305324445, 0.11870889333714613, nan, 0.0, 0.0, 0.0]
11156/1